In [17]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import warnings
import os
import MySQLdb
import pandas.io.sql as psql

In [79]:
##GETTING TELCO DATA
#I had to write a new one since jupyter notebook is not working with
#any of my saves

##HAD AN ISSUE WITH LOCAL IMPORTS, enter user and password to work



def get_db_url(comm = '!', database = '!'):
    db=MySQLdb.connect(host='!', user = '!', \
    passwd = 'RJXCMCahj151i781JMtY1T2NoIoH6KaG', db=database)
    return psql.read_sql(comm, con=db)



telco_data =get_db_url(comm = """SELECT * 
FROM customers WHERE total_charges 
NOT LIKE ' ';""", database = 'telco_churn')

telco_data = telco_data.drop(columns = ['customer_id', 'gender'])

In [82]:
##FEATURE ENGINEERING

#This is selecting the features I found to be important when checking if the data was accurate, 
#Secure count is a count of how many extra services there are while secure is a true false of if they have one
#Same when it comes to stream, and stream_count
#partner_depentents gives a specific id on what there home structure is(partner, dependents, both) 



def add_telco(data):
    ##STREAMING
    data['stream_count'] = (data['streaming_tv'].apply(lambda x : 1 if x == 'Yes' else 0)) + (data['streaming_movies'].apply(lambda x : 1 if x == 'Yes' else 0))
    data['stream'] = data['stream_count'].apply(lambda x : 1 if x > 0 else 0)
    data['techie'] = data['phone_service']
    data['fiber_support'] = (data['tech_support'] == 'Yes') & (data['internet_service_type_id'] == 2)
    ##BACK_END
    data['churn'] = data['churn'].apply(lambda x: 1 if x == 'Yes' else 0)
    data['secure_count'] = (data['online_security'].apply(lambda x : 1 if x == 'Yes' else 0)) + \
                     (data['online_backup'].apply(lambda x : 1 if x == 'Yes' else 0)) + \
                     (data['device_protection'].apply(lambda x : 1 if x == 'Yes' else 0)) + \
                     (data['tech_support'].apply(lambda x : 1 if x == 'Yes' else 0)) + \
                     (data['streaming_movies'].apply(lambda x : 1 if x == 'Yes' else 0)) + \
                     (data['streaming_tv'].apply(lambda x : 1 if x == 'Yes' else 0))
                     
    
    data['secure'] = data['secure_count'].apply(lambda x : 1 if x > 0 else 0)
    data['two_year'] = (data['contract_type_id'] == 3) & (data['tenure'] > 12)
    data['partner_dependents'] = (data['partner'].apply(lambda x: 1 if x == 'Yes' else 0)) + (data['dependents'].apply(lambda x: 2 if x == 'Yes' else 0))
    return data



In [83]:
##ENCODING TELCO


def prep_telco(data):
    encoder = LabelEncoder()
    encode_list = ['partner', 'dependents', 'online_security', 'online_backup','device_protection','tech_support','streaming_tv', \
                   'streaming_movies', 'paperless_billing', 'phone_service','multiple_lines', 'fiber_support']
    for e in encode_list:
        data[e] = encoder.fit_transform(data[e])
    return data


In [84]:
##INITIALIZING
telco_data = add_telco(telco_data)
telco_data = prep_telco(telco_data)



In [85]:
telco_baseline = len(telco_data.where(telco_data['churn'] == 0).dropna()) / len(telco_data)
print(telco_baseline)

0.7342150170648464


In [87]:
##SELECTING FEATURES
#Along with my features engineers, I used tenure, total_charges, monthly_charges, online_security, and contract_type

def tiny_telco(data):
    return data[['tenure','total_charges', 'monthly_charges', 'secure_count', \
                  'online_security', 'fiber_support', 'contract_type_id', 'churn']]
telco_data = tiny_telco(telco_data)

In [88]:
##RANDOM FOREST MODEL
#between logistical regression, decision tree, and random forest, 
#and I found the random forest to have the best accuracy and confusion matrix


x= telco_data.drop(columns = ['churn'])
y = telco_data['churn']
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 141)

logit = RandomForestClassifier(bootstrap=True, 
                            class_weight=None, 
                            criterion='gini',
                            min_samples_leaf=3,
                            n_estimators=100,
                            max_depth=3, 
                            random_state=123).fit(x_test, y_test)

y_pred = logit.predict(x_train)
y_pred_proba = logit.predict_proba(x_test)

In [89]:
##RESULTS DEFINITION STATEMENT
#this is just a copy paste of a general def statement I have to measure results

def results_train(logit, y_pred, y_pred_proba, x_train, y_train):
    print('\n\n<<<<<<<<<<<<<<<<|RESULTS|>>>>>>>>>>>>>>>\n')
    print('Accuracy of classifier on training set: {:.2f} \n'
          .format(logit.score(x_train, y_train)))
    try:
        print('Accuracy of classifier on test set: {:.2f} \n'
              .format(logit.score(x_test, y_test)))
    except:
        pass
    try:
        print('----------|FEATURE_IMPORTANCE|----------')
        print(logit.feature_importances_)
    except:
        pass
    try:
        print('Coefficient: \n', logit.coef_)
    except:
        pass
    try:
        print('Intercept: \n', logit.intercept_)
    except:
        pass
    print('')
    try:
        print('-----------|CONFUSION_MATRIX|------------')
        print(confusion_matrix(y_train, y_pred))
    except:
        pass
    try:
        print('-----------------|REPORT|-----------------')
        print(classification_report(y_train, y_pred))
        print('----------------------------------------')
    except:
        pass
    
    


In [77]:
##DISPLAY RESULTS
results_train(logit, y_pred, y_pred_proba, x_train, y_train)



<<<<<<<<<<<<<<<<|RESULTS|>>>>>>>>>>>>>>>

Accuracy of classifier on training set: 0.80 

Accuracy of classifier on test set: 0.80 

----------|FEATURE_IMPORTANCE|----------
[0.28983313 0.12300053 0.12039617 0.02414601 0.14328839 0.00372073
 0.29561505]

-----------|CONFUSION_MATRIX|------------
[[3602  270]
 [ 805  597]]
-----------------|REPORT|-----------------
              precision    recall  f1-score   support

           0       0.82      0.93      0.87      3872
           1       0.69      0.43      0.53      1402

    accuracy                           0.80      5274
   macro avg       0.75      0.68      0.70      5274
weighted avg       0.78      0.80      0.78      5274

----------------------------------------
